In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_paths = [
    './Data/BIIB.OQ.csv',
    './Data/BIO.N.csv',
    './Data/LKQ.OQ.csv',
    './Data/TXT.N.csv',
    './Data/VRSK.OQ.csv',
    './Data/MKTX.OQ.csv'
]

# Load the data
dataframes = [pd.read_csv(file) for file in file_paths]


In [3]:
processed_dfs = []
for df in dataframes:
    # Convert 'Date' to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Set the date and ticker symbol as a multi-index
    df.set_index(['Date', 'Instrument'], inplace=True)

    # Keep only the '1 Month Total Return' column
    df = df[['1 Month Total Return']]

    processed_dfs.append(df)

In [4]:
# Merge all dataframes on the multi-index
merged_data = pd.concat(processed_dfs, axis=0)

Estimate historical mean and variance covariance matrix of the stocks in your dataset. Estimate the variance covariance matrix with a Shrinkage toward Constant Correlation Approach (consider k=0.2). Estimate the vector of means with an exponential average (lambda=0.005). Comment on the correlation observed in your dataset.

In [5]:
# Define lambda parameter for exponential average
lambda_param = 0.005

# Calculate the exponential average of returns
exp_avg_returns = merged_data.groupby(level='Instrument').apply(lambda x: x.ewm(alpha=lambda_param).mean())
exp_avg_returns

1 Month Total Return
Instrument Date                      Instrument                      
BIIB.OQ    2004-01-30 00:00:00+00:00 BIIB.OQ                16.340402
           2004-02-27 00:00:00+00:00 BIIB.OQ                21.744004
           2004-03-31 00:00:00+00:00 BIIB.OQ                14.550265
           2004-04-30 00:00:00+00:00 BIIB.OQ                12.220466
           2004-05-28 00:00:00+00:00 BIIB.OQ                10.830338
...                                                               ...
VRSK.OQ    2023-07-31 00:00:00+00:00 VRSK.OQ                 1.503041
           2023-08-31 00:00:00+00:00 VRSK.OQ                 1.540937
           2023-09-29 00:00:00+00:00 VRSK.OQ                 1.508839
           2023-10-31 00:00:00+00:00 VRSK.OQ                 1.462740
           2023-11-30 00:00:00+00:00 VRSK.OQ                 1.515776

[1434 rows x 1 columns]

In [7]:
# Aggregate data to ensure unique entries for each stock per date
aggregated_data = merged_data.groupby(level=['Date', 'Instrument']).mean()

# Unstack the data after aggregation
unstacked_data = aggregated_data.unstack(level='Instrument')

# Calculate the historical variance-covariance matrix
var_cov_matrix = unstacked_data.cov()
var_cov_matrix

1 Month Total Return                        \
Instrument                                   BIIB.OQ      BIO.N     LKQ.OQ   
                     Instrument                                              
1 Month Total Return BIIB.OQ               98.491028  14.066107  13.585296   
                     BIO.N                 14.066107  62.431415  21.535464   
                     LKQ.OQ                13.585296  21.535464  74.616264   
                     MKTX.OQ               17.417318  29.516425  22.850067   
                     TXT.N                 17.396215  33.410230  38.492269   
                     VRSK.OQ               -0.871177  12.890438  12.322917   

                                                                    
Instrument                          MKTX.OQ       TXT.N    VRSK.OQ  
                     Instrument                                     
1 Month Total Return BIIB.OQ      17.417318   17.396215  -0.871177  
                     BIO.N        29.516425   33.410230  12.890438  
                     LKQ.OQ       22.850067   38.492269  12.322917  
                     MKTX.OQ     111.278210   38.155277   9.155613  
                     TXT.N        38.155277  147.855852  13.887012  
                     VRSK.OQ       9.155613   13.887012  32.816518

In [12]:
# Define the shrinkage parameter
k = 0.2

# Calculate the simple average of the variances
mean_variance = np.mean(np.diag(var_cov_matrix))

# Create a shrinkage target matrix (constant correlation matrix)
n_assets = var_cov_matrix.shape[0]
target = np.full_like(var_cov_matrix, mean_variance * (1 - k))
np.fill_diagonal(target, np.diag(var_cov_matrix))

# Apply shrinkage
shrunk_cov_matrix = k * target + (1 - k) * var_cov_matrix
shrunk_cov_matrix

1 Month Total Return                        \
Instrument                                   BIIB.OQ      BIO.N     LKQ.OQ   
                     Instrument                                              
1 Month Total Return BIIB.OQ               98.491028  25.319267  24.934618   
                     BIO.N                 25.319267  62.431415  31.294752   
                     LKQ.OQ                24.934618  31.294752  74.616264   
                     MKTX.OQ               28.000236  37.679521  32.346434   
                     TXT.N                 27.983353  40.794565  44.860196   
                     VRSK.OQ               13.369440  24.378732  23.924715   

                                                                    
Instrument                          MKTX.OQ       TXT.N    VRSK.OQ  
                     Instrument                                     
1 Month Total Return BIIB.OQ      28.000236   27.983353  13.369440  
                     BIO.N        37.679521   40.794565  24.378732  
                     LKQ.OQ       32.346434   44.860196  23.924715  
                     MKTX.OQ     111.278210   44.590603  21.390871  
                     TXT.N        44.590603  147.855852  25.175991  
                     VRSK.OQ      21.390871   25.175991  32.816518

In [13]:
# Calculate the correlation matrix
corr_matrix = unstacked_data.corr()
corr_matrix

1 Month Total Return                      \
Instrument                                   BIIB.OQ     BIO.N    LKQ.OQ   
                     Instrument                                            
1 Month Total Return BIIB.OQ                1.000000  0.179380  0.158473   
                     BIO.N                  0.179380  1.000000  0.315527   
                     LKQ.OQ                 0.158473  0.315527  1.000000   
                     MKTX.OQ                0.166496  0.350803  0.247446   
                     TXT.N                  0.144157  0.347743  0.366469   
                     VRSK.OQ               -0.015779  0.286779  0.271829   

                                                               
Instrument                        MKTX.OQ     TXT.N   VRSK.OQ  
                     Instrument                                
1 Month Total Return BIIB.OQ     0.166496  0.144157 -0.015779  
                     BIO.N       0.350803  0.347743  0.286779  
                     LKQ.OQ      0.247446  0.366469  0.271829  
                     MKTX.OQ     1.000000  0.291964  0.176924  
                     TXT.N       0.291964  1.000000  0.263755  
                     VRSK.OQ     0.176924  0.263755  1.000000